# Join: embedding, references en joins

In MongoDB kan een document andere documenten bevatten (*embedding*).
Een alternatief is het gebruik van verwijzingen naar andere documenten (*referencing*).
Als verwijzing naar een document gebruik je de `_id` (key) daarvan, als *foreign key*.

Om de gegevens uit verschillende documenten die naar elkaar verwijzen te combineren 
heb je een `join` nodig.
In veel document-databases moet je deze join zelf programmeren, in de toepassing.

MongoDB heeft (in de nieuwere versies) een join-operatie als onderdeel van de *aggregate* pijplijn.

De onderstaande opdracht is nodig als initialisatie in sommige omgevingen.

In [1]:
import os
os.environ["PATH"]=os.environ["PATH"] + ":/usr/local/bin"

## Normalisatie in relationele databases

Bij het ontwerpen van een relationele database probeer je meestal deze te normaliseren.
Deze normalisatie zorgt ervoor dat alle gegevens in de database "onafhankelijk" zijn, 
anders gezegd, dat een bepaald gegeven maar op één plaats in de database voorkomt.
Dit noemen we ook wel "single source of truth".

Een genormaliseerde vorm maakt het veranderen van de database op een consistente manier veel eenvoudiger.
Immers, als een gegeven op meerdere plaatsen voorkomt, dan moet je bij een verandering al deze verschillende plaatsen bijwerken om weer een consistente database-toestand te krijgen.

> Voorbeeld: als het adres van een persoon op 3 plaatsen in de database voorkomt, dan moet je bij een verhuizing al deze 3 plaatsen bijwerken. Als je er één of twee vergeet dat is helemaal niet meer duidelijk welk adres nu het juiste is.

Een andere reden is dat deze normalisatie, door het voorkomen van duplicaten, tot een compactere database leidt: het spaart ruimte.
Dit argument is tegenwoordig minder sterk dan vroeger, toen opslagruimte schaars en duur was.

Normalisatie heeft ook nadelen.
Het belangrijkste nadeel is dat je bij het opvragen van de data uit de database vaak één of meerdere joins nodig hebt:
je moet de waarden uit verschillende tabellen combineren tot het gevraagde resultaat.
Dit leidt bij leesopdrachten (queries) tot extra leesopdrachten voor de verschillende tabellen.

Overigens is de basisopzet van relationele databases sterk gericht op het toewijzen van data aan verschillende tabellen.
Zo moet je meervoudige relaties voorstellen met meerdere tabellen enn met verwijzingen tussen deze tabellen (via *foreign keys*), omdat een rij geen meervoudige waarden kan bevatten.
(Voorbeeld: je kunt niet een array van telefoonnummers in een rij opnemen.)

> Je kunt dit voor kleine arrays misschien oplossen door ruimte voor een paar waarden op te nemen, bijvoorbeeld voor 3 telefoonnummers. Maar dat leidt enerzijds tot beperkingen waar je later tegenaan loopt - als je er 4 nodig hebt, en anderzijds tot extra gaten in de tabel voor die rijen die deze meervoudige ruimte niet gebruiken. 

## Normalisatie in MongoDB

In MongoDB kun je in een document andere documenten *inbedden* (*embedding*).
Je kunt zelfs arrays van documenten inbedden in een document.
Dit kan leiden tot duplicatie van deeldocumenten, met het genoemde probleem van wijzigen van deze deel-documenten tot gevolg.

In plaats van inbedden (embedding) kun je ook *verwijzingen* (references) naar andere documenten gebruiken (via de `_id` van die documenten, eigenlijk een *foreign key*).

Het gevolg is dat je bij het gebruik mogelijk gegevens uit verschillende *collections* moet combineren.
Bij een SQL-database gebruik je daarvoor een join.
In MongoDB kun je daarvoor de `$loopkup`-functie gebruiken, als onderdeel van de *aggregate*-pijplijn.

Als ontwerper van de database kun zelf bepalen waar je de grens legt tussen inbedden (embedding) en verwijzingen (references).

## Voorbeeld: agenda en contacten

In een agenda-document staan (in de voorbeelden) de emailadressen van de deelnemers (`participants`) van de afspraak.
Deze kun je zien als (tijdelijke) keys van de betrokken contacten.

Deze email-adressen kunnen we vervangen door de `_id`s van deze contacten:
we gebruiken dan verwijzingen (*referencess*) tussen documenten, in plaats van *embedding*.
In zekere zin is de relatie tussen de agenda-documenten en de contacts-documenten genormaliseerd:
er is geen duplicatie van gegevens tussen deze documenten.

Zie ook: https://docs.mongodb.com/manual/core/data-modeling-introduction/

Het gebruik van verwijzingen (*references*) betekent dat we *joins* nodig hebben als we de gegevens van verschillende documenten willen combineren.
Bijvoorbeeld: als we een lijst van afspraken willen maken met daarin de namen van de deelnemers, dan moeten we daarvoor de bijbehorenden contact-documenten raadplegen.

In [2]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo

pd.set_option('max_colwidth',160)

userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
print("Database name: " + dbname)

print('Mongo version', pymongo.__version__)
client = pymongo.MongoClient('localhost', 27017)
db = client[dbname]
contacts = db.contacts
agenda = db.agenda

Database name: eelco-demodb
Mongo version 3.11.0


In [3]:
contacts.drop()
os.system('mongoimport -d ' + dbname + ' -c contacts adressen.json')

0

In [4]:
agenda.drop()
os.system('mongoimport -d ' + dbname + ' -c agenda agenda.json')

0

Merk op dat een agenda-items een lijst van deelnemers (`participants`) bevat.
In de invoer identificeren we deze deelnemers met hun emailadres.

In [5]:
list(agenda.find())

[{'_id': ObjectId('5fa9686155cba8618465aeaa'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 1, 0, 0),
  'duration': 120,
  'participants': [{'email': 'zwartejoop@ziggo.nl'}],
  'location': 'Seats2Meet Utrecht CS'},
 {'_id': ObjectId('5fa9686155cba8618465aeab'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 13, 0, 0),
  'duration': 120,
  'participants': [{'email': 'gijsbkom@ziggo.nl'},
   {'email': 'a34huis@gmail.com'}],
  'location': 'Amsterdam Zuid'},
 {'_id': ObjectId('5fa9686155cba8618465aeac'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 23, 0, 0),
  'duration': 120,
  'participants': [{'email': 'hdb@example.com'},
   {'email': 'a34huis@gmail.com'}],
  'location': 'Seats2Meet Den Bosch'},
 {'_id': ObjectId('5fa9686155cba8618465aead'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 3, 1, 0, 0),
  'duration': 60,
  'participants': [{'email': 'hdb@example.com'},
   {'email': 'a34huis@gmail.com'}],
  'location'

## Maken van *references*

In de agenda-items in de invoer gebruiken we het emailadres als de identificatie ("key") voor de deelnemers.
Voor de invoer is dit acceptabel, maar niet voor lange-termijn opslag in de database:
het emailadres van een persoon kan veranderen.
Ook de naam kun je niet als key gebruiken: namen zijn niet voldoende uniek.

Daarom willen we bij het maken/toevoegen van een afspraak het emailadres als identificatie vervangen door de interne identificatie van het contact-document: de `_id`-key.
We krijgen dan een genormaliseerde vorm gebaseerd op *references* in plaats van *embedding*.

NB: bij de update weten we hier zeker dat het document (record) al bestaat: we hoeven hier geen `$upsert` te gebruiken.

De onderstaande functie `connect` zoekt voor de deelnemers (`participants`) van het agenda-document het bijbehorende contact-document, aan de hand van het emailadres.
Vervolgens wordt als `id` veld van de deelnemer de *key* (`_id`) van het contact-document ingevuld.
Het agenda-document in de database wordt met deze gegevens bijgewerkt.

In [6]:
def connect (agendaItem):
    participants = agendaItem["participants"]
    for person in participants:
        pList = list(contacts.find({"email": person["email"]}))
        if len(pList) == 1:
            person["id"] = pList[0]["_id"]        
        else:
            print(person["email"])
            print("error in connect")
    agenda.update_one({"_id": agendaItem["_id"]}, {"$set": {"participants": participants}})

We werken alle agenda-documenten bij met behulp van `connect`: de `participants` in de agenda-documenten verwijzen dan naar de bijbehorende contact-documenten.

In [7]:
aList = list(agenda.find())
for item in aList:
    connect(item)

In het onderstaande overzicht zie je deze referenties bij de `participants`:

In [8]:
list(agenda.find())

[{'_id': ObjectId('5fa9686155cba8618465aeaa'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 1, 0, 0),
  'duration': 120,
  'participants': [{'email': 'zwartejoop@ziggo.nl',
    'id': ObjectId('5fa9686155cba8618465aea1')}],
  'location': 'Seats2Meet Utrecht CS'},
 {'_id': ObjectId('5fa9686155cba8618465aeab'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 13, 0, 0),
  'duration': 120,
  'participants': [{'email': 'gijsbkom@ziggo.nl',
    'id': ObjectId('5fa9686155cba8618465aea3')},
   {'email': 'a34huis@gmail.com', 'id': ObjectId('5fa9686155cba8618465aea2')}],
  'location': 'Amsterdam Zuid'},
 {'_id': ObjectId('5fa9686155cba8618465aeac'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 23, 0, 0),
  'duration': 120,
  'participants': [{'email': 'hdb@example.com',
    'id': ObjectId('5fa9686155cba8618465ae9e')},
   {'email': 'a34huis@gmail.com', 'id': ObjectId('5fa9686155cba8618465aea2')}],
  'location': 'Seats2Meet Den Bosch'},
 {'_i

## Join

Als we de gegevens van een agenda-document willen combineren met gegevens uit de contact-documenten, bijvoorbeeld de namen van de deelnemers, dan hebben we een *join* nodig.

In MongoDB moeten we deze join zelf programmeren.
Een join bestaat uit twee elementen:

1. het zoeken van de bijbehorende documenten (`find`);
2. het toevoegen van waarden uit een opgezocht bijbehorend document aan het resultaat-document (bijvoorbeeld, de naam uit het contact-document).

Gegeven een lijst van agenda-items, voeg hier de namen van de deelnemers aan toe.

De onderstaande functie `addNames` zoekt aan de hand van de keys (`id`) de contact-documenten van de deelnemers, en voegt de naam van elke deelnemer toe aan het agenda-document.

Voorbeeld: zoek de namen van de deelnemers bij hun emailadres in de agenda-documenten.

In [9]:
cursor = agenda.aggregate([
    {"$lookup": {"from": "contacts",
                 "localField": "participants.id",
                 "foreignField": "_id",
                 "as": "participantInfo"}}    
])
list(cursor)

[{'_id': ObjectId('5fa9686155cba8618465aeaa'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 1, 0, 0),
  'duration': 120,
  'participants': [{'email': 'zwartejoop@ziggo.nl',
    'id': ObjectId('5fa9686155cba8618465aea1')}],
  'location': 'Seats2Meet Utrecht CS',
  'participantInfo': [{'_id': ObjectId('5fa9686155cba8618465aea1'),
    'name': 'Joop de Zwart',
    'email': 'zwartejoop@ziggo.nl',
    'address': {'street': 'Rozengracht 42', 'city': 'Rotterdam'}}]},
 {'_id': ObjectId('5fa9686155cba8618465aeab'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 13, 0, 0),
  'duration': 120,
  'participants': [{'email': 'gijsbkom@ziggo.nl',
    'id': ObjectId('5fa9686155cba8618465aea3')},
   {'email': 'a34huis@gmail.com', 'id': ObjectId('5fa9686155cba8618465aea2')}],
  'location': 'Amsterdam Zuid',
  'participantInfo': [{'_id': ObjectId('5fa9686155cba8618465aea2'),
    'name': 'Adrie Vierhuis',
    'email': 'a34huis@gmail.com',
    'address': {'street': 'Beuk

We kunnen eigenlijk de oorspronkelijke `participants` vervangen door de uitgebreide informatie,
waarbij we de velden die we niet gebruiken via een projectie verwijderen:

In [10]:
cursor = agenda.aggregate([
    {"$lookup": {"from": "contacts",
                 "localField": "participants.id",
                 "foreignField": "_id",
                 "as": "participants"}},
    {"$project": {"participants.address": 0, "participants.tel": 0}}
])
list(cursor)

[{'_id': ObjectId('5fa9686155cba8618465aeaa'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 1, 0, 0),
  'duration': 120,
  'participants': [{'_id': ObjectId('5fa9686155cba8618465aea1'),
    'name': 'Joop de Zwart',
    'email': 'zwartejoop@ziggo.nl'}],
  'location': 'Seats2Meet Utrecht CS'},
 {'_id': ObjectId('5fa9686155cba8618465aeab'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 13, 0, 0),
  'duration': 120,
  'participants': [{'_id': ObjectId('5fa9686155cba8618465aea2'),
    'name': 'Adrie Vierhuis',
    'email': 'a34huis@gmail.com'},
   {'_id': ObjectId('5fa9686155cba8618465aea3'),
    'name': 'Gijs Bennekom',
    'email': 'gijsbkom@ziggo.nl'}],
  'location': 'Amsterdam Zuid'},
 {'_id': ObjectId('5fa9686155cba8618465aeac'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 23, 0, 0),
  'duration': 120,
  'participants': [{'_id': ObjectId('5fa9686155cba8618465ae9e'),
    'name': 'Hans de Boer',
    'email': 'hdb@example.com'},


**Opdracht** Pas bovenstaande aan zodat je (i) alleen de bijeenkomsten met `Fusie` als onderwerp (`subject`) krijgt; en (ii) per deelnemer (participant) alleen de naam en de woonplaats. Je mag in dit laatste geval de geneste notatie van de woonplaats laten staan.

In [11]:
cursor = agenda.aggregate([
    {"$match": {"subject": "Fusie"}},
    {"$lookup": {"from": "contacts",
                 "localField": "participants.id",
                 "foreignField": "_id",
                 "as": "participants"}},
    {"$project": {"participants.address.street": 0, "participants.address.postcode": 0, "participants.tel": 0,
                  "participants._id": 0 }}
])
list(cursor)

[{'_id': ObjectId('5fa9686155cba8618465aeae'),
  'subject': 'Fusie',
  'time': datetime.datetime(2019, 3, 19, 0, 0),
  'duration': 120,
  'participants': [{'name': 'Hans de Boer', 'email': 'hdb@example.com'},
   {'name': 'Adrie Vierhuis',
    'email': 'a34huis@gmail.com',
    'address': {'city': 'Rotterdam'}}],
  'location': 'Den Haag CS'},
 {'_id': ObjectId('5fa9686155cba8618465aeaf'),
  'subject': 'Fusie',
  'time': datetime.datetime(2019, 4, 27, 0, 0),
  'duration': 180,
  'participants': [{'name': 'Hans de Boer', 'email': 'hdb@example.com'},
   {'name': 'Adrie Vierhuis',
    'email': 'a34huis@gmail.com',
    'address': {'city': 'Rotterdam'}}],
  'location': 'Den Haag CS'}]

**Opdracht** In de `$lookup` in de voorbeelden hierboven hebben we steeds de koppeling tussen de documenten gedaan op basis van de `_id` (primaire key). Pas het vorige voorbeeld aan zodat je de koppeling doet op basis van het emailadres in beide documenten.

In [12]:
cursor = agenda.aggregate([
    {"$match": {"subject": "Fusie"}},
    {"$lookup": {"from": "contacts",
                 "localField": "participants.email",
                 "foreignField": "email",
                 "as": "participants"}},
    {"$project": {"participants.address.street": 0, "participants.address.postcode": 0, "participants.tel": 0,
                  "participants._id": 0 }}
])
list(cursor)

[{'_id': ObjectId('5fa9686155cba8618465aeae'),
  'subject': 'Fusie',
  'time': datetime.datetime(2019, 3, 19, 0, 0),
  'duration': 120,
  'participants': [{'name': 'Hans de Boer', 'email': 'hdb@example.com'},
   {'name': 'Adrie Vierhuis',
    'email': 'a34huis@gmail.com',
    'address': {'city': 'Rotterdam'}}],
  'location': 'Den Haag CS'},
 {'_id': ObjectId('5fa9686155cba8618465aeaf'),
  'subject': 'Fusie',
  'time': datetime.datetime(2019, 4, 27, 0, 0),
  'duration': 180,
  'participants': [{'name': 'Hans de Boer', 'email': 'hdb@example.com'},
   {'name': 'Adrie Vierhuis',
    'email': 'a34huis@gmail.com',
    'address': {'city': 'Rotterdam'}}],
  'location': 'Den Haag CS'}]

## Join in de toepassing

We kunnen de join ook uitprogrammeren in de toepassing, zoals in het onderstaande voorbeeld.
In het algemeen is het handiger en efficiënter om de ingebouwde database-opdrachten te gebruiken.
Maar, zoals gezegd, niet elke document-database heeft een ingebouwde join-operatie.

Een join bestaat uit twee elementen:

1. het zoeken van de bijbehorende documenten (`find`);
2. het toevoegen van waarden uit een opgezocht bijbehorend document aan het resultaat-document (bijvoorbeeld, de naam uit het contact-document).

> In de aggregate-pijplijn zijn dit de `$match` en `$lookup`-stappen.

Gegeven een lijst van agenda-items, voeg hier de namen van de deelnemers aan toe.

De onderstaande functie `addNames` zoekt aan de hand van de keys (`id`) de contact-documenten van de deelnemers, en voegt de naam van elke deelnemer toe aan het agenda-document.

In [13]:
def addNames(agendaItem):
    participants = agendaItem["participants"]
    for person in participants:
        pList = list(contacts.find({"_id": person["id"]})) ## (1) find
        if len(pList) == 1:
            person["name"] = pList[0]["name"]  ## (2) toevoegen van naam   
        else:
            print(person["email"])
            print("error in addName")

We voeren hier de genoemde *join* uit voor alle agenda-documenten.
In de praktijk voer je eerst een selectie uit op de agenda-documenten die gebruikt worden;
alleen voor die documenten voer je dan de join uit.

In [14]:
aList = list(agenda.find())
for item in aList:
    addNames(item)
aList

[{'_id': ObjectId('5fa9686155cba8618465aeaa'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 1, 0, 0),
  'duration': 120,
  'participants': [{'email': 'zwartejoop@ziggo.nl',
    'id': ObjectId('5fa9686155cba8618465aea1'),
    'name': 'Joop de Zwart'}],
  'location': 'Seats2Meet Utrecht CS'},
 {'_id': ObjectId('5fa9686155cba8618465aeab'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 13, 0, 0),
  'duration': 120,
  'participants': [{'email': 'gijsbkom@ziggo.nl',
    'id': ObjectId('5fa9686155cba8618465aea3'),
    'name': 'Gijs Bennekom'},
   {'email': 'a34huis@gmail.com',
    'id': ObjectId('5fa9686155cba8618465aea2'),
    'name': 'Adrie Vierhuis'}],
  'location': 'Amsterdam Zuid'},
 {'_id': ObjectId('5fa9686155cba8618465aeac'),
  'subject': 'Beleidsplan',
  'time': datetime.datetime(2019, 4, 23, 0, 0),
  'duration': 120,
  'participants': [{'email': 'hdb@example.com',
    'id': ObjectId('5fa9686155cba8618465ae9e'),
    'name': 'Hans de Boer'},
   {

### Opdracht
Programmeer nu de onderstaande opdracht zelf uit.

**Opdracht**
Maak een lijst van agenda-documenten voor de "Beleidsplan"-bijeenkomsten, met daarin als onderdeel van de `participants` de namen en woonplaatsen van de deelnemers.

Splits dit in 3 stappen:

1. definitie van een functie `joinContacts(agendaItem)`
2. construeren van de lijst met "Beleidsplan"-bijeenkomsten
3. het toepassen van de genoemde functie op alle elementen in deze lijst.

Houd er bij stap 1 rekening mee dat het veld "address.city" mogelijk niet bestaat.

In [15]:
def joinContacts(agendaItem):
    participants = agendaItem["participants"]
    for person in participants:
        pList = list(contacts.find({"_id": person["id"]})) ## (1) find
        if len(pList) == 1:
            person["name"] = pList[0]["name"]  ## (2) toevoegen van naam
            if "address" in pList[0] and "city" in pList[0]["address"]:
              person["city"] = pList[0]["address"]["city"]
        else:
            print(person["email"])
            print("error in addName")

In [16]:
bList = list(agenda.find({"subject": "Fusie"}))
for item in bList:
    joinContacts(item)
bList

[{'_id': ObjectId('5fa9686155cba8618465aeae'),
  'subject': 'Fusie',
  'time': datetime.datetime(2019, 3, 19, 0, 0),
  'duration': 120,
  'participants': [{'email': 'hdb@example.com',
    'id': ObjectId('5fa9686155cba8618465ae9e'),
    'name': 'Hans de Boer'},
   {'email': 'a34huis@gmail.com',
    'id': ObjectId('5fa9686155cba8618465aea2'),
    'name': 'Adrie Vierhuis',
    'city': 'Rotterdam'}],
  'location': 'Den Haag CS'},
 {'_id': ObjectId('5fa9686155cba8618465aeaf'),
  'subject': 'Fusie',
  'time': datetime.datetime(2019, 4, 27, 0, 0),
  'duration': 180,
  'participants': [{'email': 'hdb@example.com',
    'id': ObjectId('5fa9686155cba8618465ae9e'),
    'name': 'Hans de Boer'},
   {'email': 'a34huis@gmail.com',
    'id': ObjectId('5fa9686155cba8618465aea2'),
    'name': 'Adrie Vierhuis',
    'city': 'Rotterdam'}],
  'location': 'Den Haag CS'}]

### Opmerkingen

#### Meer gegevens per *participant*

De gegevens van de deelnemers in de afspraak kunnen we later uitbreiden, bijvoorbeeld met de opmerking of ze al uitgenodigd zijn, en of ze al bevestigd hebben.
Dat valt buiten het bestek van dit voorbeeld.

#### Gedeeltelijke de-normalisatie

In het voorbeeld hierboven kun je de namen van de deelnemers ook in het agenda-document opnemen, naast de `id` van het contact-document.
Op die manier hoef je niet steeds een join uit te voeren met de contact-documenten als je alleen de namen vn de deelnemers nodig hebt.
Er is dan wel sprake van *duplicatie* van gegevens: de naam van een contact komt zowel in het contact-document als in meerdere agenda-documenten voor.
Als de naam van een contact niet verandert, levert dit geen probleem met inconsistenties op.
Een klein nadeel is dat de opslag wat minder compact is, maar dat weegt niet op tegen het voordeel van snellere toegang tot de gegevens

> Duplicatie van gegevens versnelt in het algemeen het opzoeken van gegevens,
  maar kan het aanpassen van gegegevens (veel) lastiger maken.
  Voor gegevens die niet veranderen is duplicatie geen probleem.

#### Caching

De aanpak in de functie `addNames` is niet de meest efficiënte:
eenzelfde contact-verwijzing komt meerdere malen voor, we zoeken dit document steeds opnieuw op.
Dit kunnen we efficiënter maken door een *cache* toe te voegen van personen die we al eerder opgezocht hebben.
Dat vraagt maar een kleine aanpassing in het algoritme.

> Toevoegen: voorbeeld van caching

#### Linked data

In het voorbeeld van de agenda gebruiken we het emailadres als identificatie van een persoon, of, later, de interne key van het contact-document.
Bij Linked Data zullen we nog andere manieren tegenkomen voor het identificeren van een persoon (contact).

#### Embedding versus referencing

Bij MongoDB kun je kiezen tussen embedding en referencing.
Wat zijn hierbij goede criteria?

* gebruik altijd *referencing* voor documenten die ook zelfstandig voor kunnen komen, en zie zelfstandig aangemaakt, veranderd en verwijderd kunnen worden, zoals in het bovenstaande voorbeeld agenda-documenten en contact-documenten.
    * eventueel kun je deze referencing uitbreiden met een gedeeltelijke embedding van niet-veranderende velden (zoals de naam).
* gebruik *embedding* voor gegevens die bij elkaar horen, en die zelfstandig geen betekenis hebben. In het bovenstaande voorbeeld is het veld `participants` daarvan een voorbeeld.
